In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Load results from WeatherPy

In [2]:
# Loading in csv from WeatherPy activity into a DataFrame
weather_path = '../WeatherPy/Output/cities.csv'
weather_df = pd.read_csv(weather_path)

## Humidity Heatmap

In [3]:
# Configuring gmaps with the api key
gmaps.configure(api_key=g_key)

# Layout of heatmap figures
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [4]:
# Storing latitude and longitude in a DataFrame as floats
locations = weather_df[['Lat', 'Lng']].astype(float)
# Storing humidity for weight of heatmap
humidity = weather_df['Humidity'].astype(float)

In [5]:
# Creating heatmap of all locations 
fig = gmaps.figure(figure_layout)
# Heatmap layer weighted based on humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=100, point_radius=1)
# Applying heatmap layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create a new DataFrame fitting weather criteria
Ideal Weather Condition: Wind Speed < 7 mph, max temperature $\leq$ 75°F, humidity < 40, and cloudiness < 25

In [6]:
# Keeping only the rows with the ideal weather conditions
ideal_weather = weather_df.loc[(weather_df['Wind Speed'] < 7) &
                               (weather_df['Max Temp'] <= 75) &
                               (weather_df['Humidity'] < 40) &
                               (weather_df['Cloudiness'] < 25)]

# Dropping na rows
ideal_weather = ideal_weather.dropna()

ideal_weather.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,8.000000,8.000000,8.000000,8.000000,8.00000,8.000000,8.000000,8.000000e+00
mean,397.250000,33.377088,-7.753737,62.031250,29.12500,8.750000,3.495000,1.651299e+09
std,171.812813,7.783191,105.099887,10.172131,10.35702,10.911986,1.390118,1.681559e+02
min,137.000000,18.385100,-115.530300,48.020000,12.00000,0.000000,1.230000,1.651298e+09
25%,315.500000,31.454850,-109.552775,51.402500,22.25000,0.000000,2.820000,1.651299e+09
50%,405.000000,34.650000,-15.453500,67.550000,32.50000,2.500000,3.510000,1.651299e+09
75%,558.500000,36.931450,80.437500,70.342500,37.50000,19.250000,4.205000,1.651299e+09
max,575.000000,42.800000,135.583300,70.650000,39.00000,24.000000,5.750000,1.651299e+09


## Hotel Map

In [7]:
# Extracting the city, country, latitude, and longitude from the ideal weather DataFrame
hotel_df = pd.DataFrame({'City': ideal_weather['City'], 'Country': ideal_weather['Country'],
                        'Lat': ideal_weather['Lat'], 'Lng': ideal_weather['Lng']})
                             
# Adding a blank column for hotel names
hotel_df['Hotel Name'] = ''

hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Westport,US,41.1415,-73.3579,
1,Kargil,IN,34.5667,76.1000,
2,Faya,SA,18.3851,42.4509,
3,Kadoma,JP,34.7333,135.5833,
4,Brawley,US,32.9787,-115.5303,
5,Gallup,US,35.5281,-108.7426,
6,Kumul,CN,42.8000,93.4500,
7,Mulegé,MX,26.8833,-111.9833,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Parameter dictionary for nearby search
params = {
    'location': '',
    'type': 'lodging',
    'radius': 5000,
    'key': g_key
}

# Using iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # Updating parameters with latitude and longitude for each city
    params['location'] = f'{row["Lat"]}, {row["Lng"]}'
    
    # Saving the request for each city as a json
    response = requests.get(base_url, params=params).json()
    
    # Assigning the name of the first hotel for each city in Hotel Name column
    try:
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
    except:
        # If there is none, then pass the city
        pass

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Westport,US,41.1415,-73.3579,The Inn at Longshore
1,Kargil,IN,34.5667,76.1000,Hotel Caravan Sarai Kargil
2,Faya,SA,18.3851,42.4509,املاك خالد الشعشوع
3,Kadoma,JP,34.7333,135.5833,Hotel Agora Osaka Moriguchi
4,Brawley,US,32.9787,-115.5303,Brawley Inn Hotel & Conference Center
5,Gallup,US,35.5281,-108.7426,Quality Inn & Suites Gallup I-40 Exit 20
6,Kumul,CN,42.8000,93.4500,Laoyutang Resort
7,Mulegé,MX,26.8833,-111.9833,Casa Granada


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))